# VIX Predictor using an adaBoost Model

In [663]:
# Import appropriate modules

import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

import datetime
import numpy as np
import yfinance as yf
from datetime import datetime
from pandas.tseries.offsets import DateOffset
import hvplot
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from vix_functions import garch_fit_and_predict, correlation_filter, retrieve_yahoo_close, retrieve_yahoo_volume

from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

# CONTROL PANEL

In [664]:
# Key parameters of the model

# Min return to set up a positive signal
threshold= 0.00 

# Split of data
training_period_months = 120

# Adaboost parameters
adaboost_estimators = 6
learning_rate_adaboost = 1
max_depth=2


# Inclusion of the first 4 components lag1
#n: number of components to include
number_of_pca_lag_component_to_include = 4
num_pca_components = 40


# Definition of demo mode or development mode
demo_mode = False
parameter_tuning_mode = True
run_multiple_tuning_iterations = True

# Generation of the Features Matrix X

### X1: close prices
#### 40 units: 
    * Close prices of international indexes of stocks and bonds, 
    * key stocks, 
    * currency exchange rates, 
    * commodities 

In [674]:
# Ticker List: VIX must be in first position
# ticker_list_ORIGINAL= ["^VIX", "spy", 'XLF', 'XLE',
#               'EURUSD=X', 'GBPUSD=X', 'AUDUSD=X', 'BRLUSD=X', "DX-Y.NYB","USDJPY=X", 
#               '^TNX', 'ZB=F', 'ZF=F', 'NQ=F','NKD=F',                                       
#               'LQD',
#               'AAPL', 'AMZN', 'GE','MU','MSFT', 'BMY', 'FDX', 'GS','PLD','NVDA',   
#               "tlt", "ief", 
#               "FXI", "EZU", "EEM", "EFA", 'FEZ', "^GDAXI", '^FTSE','^HSI','^FCHI',              #'^GSPC',
#               "gld", "slv", "CL=F"]

ticker_list= ["^VIX", "spy", 
               'GBPUSD=X', 'BRLUSD=X', "DX-Y.NYB","USDJPY=X", 
              '^TNX', 'ZF=F', 'NQ=F','NKD=F',                                       
              'LQD',
              'AAPL', 'AMZN','MU','MSFT', 'BMY', 'PLD', 
              "FXI", 'FEZ', '^FTSE','^FCHI','^GSPC',
              "gld", "slv", 
              "ES=F","QM=F", "BIO=F"
             ]



    
# Some of the less familiar tickers are listed below
# CAC 40 (^FCHI)
# Yen Denominated TOPIX Futures,D (TPY=F)
# FTSE 100 (^FTSE)
# SPDR EURO STOXX 50 ETF (FEZ)
# DAX PERFORMANCE-INDEX (^GDAXI)
# S&P500 Index (^GSPC)
# HANG SENG INDEX (^HSI)
# 13 Week Treasury Bill (^IRX) --eliminate
# Nikkei/USD Futures,Dec-2021 (NKD=F)
# iShares iBoxx $ Investment Grade Corporate Bond ETF (LQD)
# Nasdaq 100 Dec 21 (NQ=F)
# NVIDIA Corporation (NVDA)
# Euro spot  'EURUSD'
# American Funds U.S. Government Securities Fund Class C (UGSCX) - 2001
# Micron Technology, Inc. (MU)
# Microsoft Corporation (MSFT)
# Bristol-Myers Squibb Company (BMY)
# FEDEX CORP (FDX)
# The Goldman Sachs Group, Inc. (GS)
# Prologis, Inc. (PLD)
# Treasury Yield 10 Years (^TNX) -- 1985
# Energy Select Sector SPDR Fund (XLE)
# Financial Select Sector SPDR Fund (XLF)
# U.S. Treasury Bond Futures,Dec- (ZB=F) - 2000
# Five-Year US Treasury Note Futu (ZF=F) - 2000
# E-Mini S&P500 (ES=F)
# E-Mini oil (QM=F)
# E-MinI Nasdaq (BIO=F)
print(f"Current number of tickers: {len(ticker_list)}")


Current number of tickers: 27


In [675]:
# X1: Upload of data using API
def retrieve_close(close_prices_dict, ticker_list):
    """
    This function retrieves close prices from Yahoo Finance
    
    Arg:
    close_prices_dict: empty dictionary to be filled with the close prices
    ticker_list: a list of tickers of the yahoo close prices to be retrieve
    
    Return:
    A dictionary with close prices
    """
    
    
    for ticker in ticker_list:
        close_price = retrieve_yahoo_close(ticker, start_date='2006-07-02', end_date='2021-11-17')
        close_prices_dict[ticker] = close_price
    return close_prices_dict


if demo_mode == True:
    close_prices_df = pd.read_csv("adaboost_close_prices.csv", index_col="Date", parse_dates=True, infer_datetime_format=True)
else:
    close_prices_dict = {}
    close_prices_dict = retrieve_close(close_prices_dict, ticker_list)
    close_prices_df= pd.DataFrame(close_prices_dict)
    close_prices_df.to_csv("adaboost_close_prices.csv", index=True)
print("Completed retrieve of close prices")

Processing Close ^VIX
Processing Close spy
Processing Close GBPUSD=X
Processing Close BRLUSD=X
Processing Close DX-Y.NYB
Processing Close USDJPY=X
Processing Close ^TNX
Processing Close ZF=F
Processing Close NQ=F
Processing Close NKD=F
Processing Close LQD
Processing Close AAPL
Processing Close AMZN
Processing Close MU
Processing Close MSFT
Processing Close BMY
Processing Close PLD
Processing Close FXI
Processing Close FEZ
Processing Close ^FTSE
Processing Close ^FCHI
Processing Close ^GSPC
Processing Close gld
Processing Close slv
Processing Close ES=F
Processing Close QM=F
Processing Close BIO=F
Completed retrieve of close prices


In [676]:
# X1 Fill of missing values
close_prices_df=close_prices_df.ffill(axis='rows')
close_prices_df=close_prices_df.bfill(axis='rows')

# Apply correlation filter to keep just series with some minimum level of correlation of 0.2
close_prices_component_df = correlation_filter(close_prices_df, min_corr=0.20, key_column='^VIX', eliminate_first_column=True)

X1=close_prices_component_df
vix=close_prices_df['^VIX']
vix_ret=close_prices_df['^VIX'].pct_change()
VIX=pd.DataFrame([vix, vix_ret]).T
VIX.columns=['VIX','VIX_ret']

X1_no_suffix=pd.concat([VIX,close_prices_component_df], axis=1)

X1=X1_no_suffix.add_suffix("_close")

print("Completed inclusion of close prices")

Completed inclusion of close prices


In [677]:
# Presentation graphs of the SPY and the VIX

presentation_graph=pd.concat([X1['spy_close'], 2.5*X1['VIX_close']],axis=1).hvplot(
        title='VIX and S&P (Scale adjusted)',
        width=1000
)
presentation_graph

graph1=X1['spy_close'].hvplot(
                title= "SPY: iShares S&P 500 ETF Close Price",
                ylabel= 'Close Price [$]'
) 

graph2=X1['VIX_close'].hvplot(
                color ='red',
                title ='VIX: CBOE Volatility Index',
                ylabel= '[%]'

)
display(graph1)
graph2 



:Curve   [Date]   (spy_close)

:Curve   [Date]   (VIX_close)

### X2: returns

In [678]:
# Inclusion of security returns X2
# Include returns that are correlated more than 0.20 with the Vix return

security_returns_df= close_prices_df.pct_change()
security_returns_component_df = correlation_filter(
                                        security_returns_df, 
                                        min_corr=0.20, 
                                        key_column='^VIX', 
                                        eliminate_first_column=True 
)

X2_no_suffix=security_returns_component_df

X2=X2_no_suffix.add_suffix("_returns")

print("Completed inclusion of returns")

Completed inclusion of returns


### X3: volume

In [679]:
# inclusion of security volume X3
volume_list = ticker_list[1:len(ticker_list)]

def retrieve_volume(volume_dict, volume_list):
    """
    This function retrieve volume trades from a list of tickers
    
    Args:
    volume_dict: an initial dictionary to populate
    volume_list: the list of tickers for which to retrieve the volume
    
    
    Return:
    The original dictionary filled with the volume of the list of tickers
    """
    for ticker in volume_list:        
        volume = retrieve_yahoo_volume(ticker)
        volume_dict[ticker] = volume
    return volume_dict

if demo_mode == True:
    volume_df = pd.read_csv("adaboost_volume.csv", index_col="Date", parse_dates=True, infer_datetime_format=True)
else:
    volume_dict = {}
    volume_dict = retrieve_volume(volume_dict, volume_list)
    volume_df= pd.DataFrame(volume_dict)
    volume_df.to_csv("adaboost_volume.csv", index=True)
print("Completed retrieve of volume")

volume_df_with_vix=pd.concat([vix, volume_df], axis=1)
#print(volume_df_with_vix.corr())

volume_component_df = correlation_filter(volume_df_with_vix, min_corr=0.20, key_column='^VIX', eliminate_first_column=True )
X3_no_suffix=volume_component_df

X3=X3_no_suffix.add_suffix("_volume")

Processing Volume spy
Processing Volume GBPUSD=X
Processing Volume BRLUSD=X
Processing Volume DX-Y.NYB
Processing Volume USDJPY=X
Processing Volume ^TNX
Processing Volume ZF=F
Processing Volume NQ=F
Processing Volume NKD=F
Processing Volume LQD
Processing Volume AAPL
Processing Volume AMZN
Processing Volume MU
Processing Volume MSFT
Processing Volume BMY
Processing Volume PLD
Processing Volume FXI
Processing Volume FEZ
Processing Volume ^FTSE
Processing Volume ^FCHI
Processing Volume ^GSPC
Processing Volume gld
Processing Volume slv
Processing Volume ES=F
Processing Volume QM=F
Processing Volume BIO=F
Completed retrieve of volume


### X4: GARCH Models
    * GJR-GARCH Model conditional volatility
    * Model a response to shocks
    * Allow an asymmetryc t-student distribution 

In [680]:
# Inclusion of GARCH series X4
garch_series=pd.DataFrame()

not_to_include=['^VIX', 'BIO=F']


for ticker in ticker_list:
    
        if ticker in not_to_include:
            continue
    
        if demo_mode == True:
            print_series = False
        else:
            print_series = True
        garch_series[ticker]=garch_fit_and_predict(security_returns_df[ticker], ticker, horizon=1, p=1, q=1, o=1, print_series_name=print_series)
            
        
X4_no_suffix=garch_series
X4=X4_no_suffix.add_suffix("_garch")

if demo_mode == False:
    X4

print('GARCH Process fit and predictions completed')


Processing series: spy.....
Processing series: GBPUSD=X.....
Processing series: BRLUSD=X.....
Processing series: DX-Y.NYB.....
Processing series: USDJPY=X.....
Processing series: ^TNX.....
Processing series: ZF=F.....
Processing series: NQ=F.....
Processing series: NKD=F.....
Processing series: LQD.....
Processing series: AAPL.....
Processing series: AMZN.....
Processing series: MU.....
Processing series: MSFT.....
Processing series: BMY.....
Processing series: PLD.....
Processing series: FXI.....
Processing series: FEZ.....
Processing series: ^FTSE.....
Processing series: ^FCHI.....
Processing series: ^GSPC.....
Processing series: gld.....
Processing series: slv.....
Processing series: ES=F.....
Processing series: QM=F.....
GARCH Process fit and predictions completed


### X5: Return squared


In [458]:
# Inclusion of return squares in X5

returns_squared_df_no_vix= security_returns_df.drop(columns='^VIX')**2
returns_squared_and_vix_level_df=pd.concat([vix,returns_squared_df_no_vix], axis=1)
returns_squared_component_df = correlation_filter(returns_squared_and_vix_level_df, min_corr=0.20, key_column='^VIX', eliminate_first_column=True)

X5_no_suffix_df=returns_squared_component_df
X5=X5_no_suffix_df.add_suffix("_return_squared")

if demo_mode == False:
    X5
print("Completed inclusion of return squared")

Completed inclusion of return squared


### X6: Google Trends

In [459]:
# # Upload of Google Tremds -- X6
# keywords=['banking', "consumer", "depression", "gdp", "inflation",
#           'unemployment', 'liquidity','cci', 'vix_word']
# google_trends_df=pd.DataFrame()

# for keyword in keywords:
#     file_path=f"./Resources/Google_trends/{keyword}.csv"
#     if demo_mode == False:
#         print(file_path)
#     trend=pd.read_csv(Path(file_path),
#                       index_col= 'Daily', 
#                       parse_dates= True,
#                       infer_datetime_format=True
#                      )
#     #print(trend)
#     google_trends_df=pd.concat([google_trends_df, trend], axis=1)
#     #print(google_trends_df)

# if demo_mode == False:
#     google_trends_df
# print('Google Trends load completed')

In [460]:
# # Working on preparing Google-trends data

# # Unifying google dates with VIX
# minimum_date=vix.index.min()
# maximum_date=vix.index.max()

# google_trends_df=google_trends_df.loc[minimum_date:maximum_date,:]
# #print(google_trends_df.iloc[0,:])

# vix_google_trends_df=pd.concat([vix, google_trends_df], axis=1)
# vix_google_trends_df.isna().sum()

# #print(vix_google_trends_df.head())

# #vix_google_trends_df=vix_google_trends_df.fillna(0)
# #vix_google_trends_df
# #vix_google_trends_df.loc[vix_google_trends_df['^VIX'].isna(),['^VIX','Banking: (United States)']]

# # We will drop Saturday Sunday, but we would like to average Fri-Sat-Sun and reset the value of Friday
# vix_google_trends_df=vix_google_trends_df.dropna()
# google_trends_df=vix_google_trends_df.iloc[:,1:]
# #google_trends_df.isna().sum()

In [461]:
# # Filtering by correlation X6

# google_trends_component_df = correlation_filter(
#                                 vix_google_trends_df, 
#                                 min_corr=0.05, 
#                                 key_column='^VIX', 
#                                 eliminate_first_column=True)

# X6=google_trends_component_df

# # We will interpolate so we can fill the missing data only on Google Trends
# pro_interpolation_of_X6=pd.concat([vix, X6], axis=1)
# pro_interpolation_of_X6=pro_interpolation_of_X6.interpolate(method="polynomial", order=2, axis=0)
# pro_interpolation_of_X6
# X6_no_suffix_df = pro_interpolation_of_X6.iloc[:,1:]
# X6 = X6_no_suffix_df.add_suffix("_google_trends")

# if demo_mode == False:
#     X6.shape
# print('Google Trends process inclusion completed')

### X7: Economic and Financial Series

In [462]:
# #Economic Series
# # Upload of csv files -- X7
# keywords=['JobClaimsWeeklySeries', 'vix_put_call_ratio']
# economic_series_df=pd.DataFrame()

# for keyword in keywords:
#     file_path=f"./Resources/Economic_and_financial_Series/{keyword}.csv"
#     if demo_mode == False:
#         print(file_path)
#     new_series=pd.read_csv(Path(file_path),
#                       index_col= 'DATE', 
#                       parse_dates= True,
#                       infer_datetime_format=True
#                      )
#     new_series=new_series.iloc[:,0]
#     if keyword=='JobClaimsWeeklySeries':
#         new_series=new_series.shift(-1, freq='D')
#     if demo_mode == False:
#         print(new_series)
#     # Adjustment due to weekend data. We are going to assign data on the weekends to Friday, since are going to be 
#     # consider for the the prediction of Monday
#     economic_series_df=pd.concat([economic_series_df, new_series], axis=1)
#     #print(economic_series_df.tail())

# economic_series_df
# economic_series_change_df = economic_series_df.pct_change().add_suffix('_change')

# if demo_mode == False:
#     economic_series_df.loc[:,:].tail(20)

In [463]:
# # Preparation of economic variables

# # Changes of columns that are on a weekend - concat with vix to add week days
# vix_economic= pd.concat([vix,economic_series_df,economic_series_change_df ],axis=1)
# vix_economic['VIX Put/Call Ratio']= vix_economic['VIX Put/Call Ratio'].fillna(0)

# # Applying interpolation to appropiate columns. Levels: interpolation, changes: zeros
# vix_economic.loc[:,economic_series_df.columns]=vix_economic.loc[:,economic_series_df.columns].interpolate(method="polynomial", order=2, axis=0)
# vix_economic.loc[:,economic_series_change_df.columns]=vix_economic.loc[:,economic_series_change_df.columns].fillna(0)

# #print(vix_economic)

# #Filtering for available dates
# economic_series_ready_df = vix_economic.loc[minimum_date:maximum_date,:]
# economic_series_ready_df = economic_series_ready_df.iloc[:,1:]

# X7_no_suffix_df=economic_series_ready_df
# X7=X7_no_suffix_df.add_suffix("_macroeconomics")

# print("Completed inclusion of economic variables")

### X8: volatility of the SPY in several rolling windows

In [695]:
# SPY volatility on varios tracks X8

#if demo_mode == True:
#    close_price_spy_df = pd.read_csv("adaboost_spy_data.csv", index_col="Date", parse_dates=True, infer_datetime_format=True)
    
# Retrieve yahoo close prices for spy, so to be able to calculate rolling volatilities
#else:
close_price_spy_df = retrieve_yahoo_close('spy', start_date='2005-01-01', end_date='2021-11-17')  #MATCH DATE ABOVE
close_price_spy_df.to_csv("adaboost_spy_data.csv", index=True)
    
# Calculate returns
spy_returns_df=close_price_spy_df.pct_change()

#Initialize dataframe for volatility
spy_volatility=pd.DataFrame()

# Define rolling windows to create
windows_for_lag=[10,20,30,60,90,120,180,200,260]

# Loop to create the volatilities
for window_size in windows_for_lag:
    column_name=f"{window_size}_spy_rolling_volatility"
    spy_volatility[column_name] = spy_returns_df.rolling(window=window_size).std()

# Concatenate to vix to uniform index
X8=pd.concat([vix, spy_volatility], axis=1)

# Define min and max values for the window
X8=X8.loc[minimum_date:maximum_date,:]        ## CORRECT MAXIMUM DATE
    
# Fill missing data
X8=X8.ffill()
X8=X8.iloc[:,1:]

# Setting for demo
if demo_mode == False:
    X8.shape
print("Inclusion of rolling volatilities completed")
X8.tail()

Processing Close spy
Inclusion of rolling volatilities completed


Timestamp('2021-10-01 00:00:00', freq='B')

### X9: Technical Indicators

In [465]:
# # Inclusion of Technical Indicators
# technical_indicators = pd.read_csv("technical_indicators.csv", index_col="Date", parse_dates=True, infer_datetime_format=True)
# technical_indicators = technical_indicators.drop(columns=["vix close", "vix return", "mean"])
# X9 = pd.concat([security_returns_df['spy'], technical_indicators], axis=1)
# X9=X9.loc[minimum_date:maximum_date,:]
# if demo_mode == False:
#     display(X9.shape)
# X9=X9.ffill()
# X9=X9.iloc[:,1:]


# if demo_mode == False:
#     display(X9.shape)
# # display(X9.isna().sum())
# print("Completed inclusion of technical indicators")

(3980, 37)

(3980, 36)

Completed inclusion of technical indicators


### X10: Day of the Week effect

In [685]:
#Inclusion of day of week data
day_of_week = pd.read_csv("prophet_output_day_of_week.csv", index_col="Date", parse_dates=True, infer_datetime_format=True)
day_of_week = day_of_week.drop(columns=["y"])
X10 = pd.concat([security_returns_df['spy'], day_of_week], axis=1)
X10 = X10.loc[minimum_date:maximum_date,:]
if demo_mode == False:
    display(X10.shape)
X10 = X10.ffill()
X10 = X10.iloc[:,1:]
if demo_mode == False:
    display(X10.shape)
#display(X10.isna().sum())
print("Completed inclusion of day of the week effect")
X10.tail(20)

(3980, 6)

(3980, 5)

Completed inclusion of day of the week effect


,Friday,Monday,Thursday,Tuesday,Wednesday
Date,,,,,
2021-09-06,1.0,0.0,0.0,0.0,0.0
2021-09-07,0.0,0.0,0.0,1.0,0.0
2021-09-08,0.0,0.0,0.0,0.0,1.0
2021-09-09,0.0,0.0,1.0,0.0,0.0
2021-09-10,1.0,0.0,0.0,0.0,0.0
2021-09-13,0.0,1.0,0.0,0.0,0.0
2021-09-14,0.0,0.0,0.0,1.0,0.0
2021-09-15,0.0,0.0,0.0,0.0,1.0
2021-09-16,0.0,0.0,1.0,0.0,0.0


### X11: Neural Netework prediction

In [467]:
# #Inclusion of neural network data
# predictions_train_test_df = pd.read_csv("neural_network_output.csv", index_col="Date", parse_dates=True, infer_datetime_format=True)
# X11 = pd.concat([security_returns_df['spy'], predictions_train_test_df], axis=1)
# X11 = X11.loc[minimum_date:maximum_date,:]
# if demo_mode == False:
#     display(X11.shape)
# X11 = X11.ffill()
# X11 = X11.fillna(0)
# X11 = X11.iloc[:,1:]
# if demo_mode == False:
#     display(X11.shape)
# print("Completed inclusion of the Neural Network prediction")

# GENERATION OF THE FEATURE MATRIX **X**

In [697]:
# Concatenation of all sources of data
XY=pd.concat([X1, X2, X3, X4, X5, X8, X10  ], axis=1) # X6,X7,X9 X11
if parameter_tuning_mode == True:
    print(XY.shape)

XY.dropna(subset = ['VIX_close', 'VIX_ret_close'])   ###DO WE WANT TO TAKE IT OUT??
if parameter_tuning_mode == True:
    print(XY.shape)

# Interpolation is not applied to numerical variables. We are just going to drop those.
print(f"XY.shape: {XY.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X1.shape: {X1.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X2.shape: {X2.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X3.shape: {X3.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X4.shape: {X4.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X5.shape: {X5.shape}, {XY.index.min()}, {XY.index.max()} ")
#print(f"X6.shape: {X6.shape}, {XY.index.min()}, {XY.index.max()} ")
#print(f"X7.shape: {X7.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X8.shape: {X8.shape}, {XY.index.min()}, {XY.index.max()} ")
#print(f"X9.shape: {X9.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X10.shape: {X10.shape}, {XY.index.min()}, {XY.index.max()} ")
#print(f"X11.shape: {X11.shape}, {XY.index.min()}, {XY.index.max()} ")

#display(XY.isna().head(40))
#display(XY.isna().sum().tail(40))
#XY=XY.dropna()
if parameter_tuning_mode == True:
    XY.shape
    XY.head()

(4013, 108)
(4013, 108)
XY.shape: (4013, 108), 2006-07-03 00:00:00, 2021-11-17 00:00:00 
X1.shape: (4013, 14), 2006-07-03 00:00:00, 2021-11-17 00:00:00 
X2.shape: (4013, 18), 2006-07-03 00:00:00, 2021-11-17 00:00:00 
X3.shape: (4013, 14), 2006-07-03 00:00:00, 2021-11-17 00:00:00 
X4.shape: (4012, 25), 2006-07-03 00:00:00, 2021-11-17 00:00:00 
X5.shape: (4013, 23), 2006-07-03 00:00:00, 2021-11-17 00:00:00 
X8.shape: (3980, 9), 2006-07-03 00:00:00, 2021-11-17 00:00:00 
X10.shape: (3980, 5), 2006-07-03 00:00:00, 2021-11-17 00:00:00 


# y:  Set the Signal

In [698]:
# Set the Signal column
vix_ret=vix.pct_change()
XY["Signal"] = 0.0

XY.loc[(XY['VIX_ret_close'] >= threshold), 'Signal'] = 1

# # Generate the trading signals 1 (entry) or -1 (exit)
# # where 1 is when the ^VIX is greater than 3.6%.
# # where 0 is when the ^VIX  is less than 3.6%.
#for index, row in XY.iterrows():
#    if row["VIX_ret"] >= 0.036:
#        XY.loc[index, "Signal"] = 1.0

# Review the DataFrame
if parameter_tuning_mode == True:
    print(XY["Signal"].head())
    XY["Signal"].value_counts()
XY.loc[XY["Signal"]==1, 'VIX_ret_close']
#XY.shape  ## 3981
XY.loc['2007-07-13':,'VIX_ret_close']

Date
2006-07-03    0.0
2006-07-04    1.0
2006-07-05    1.0
2006-07-06    0.0
2006-07-07    1.0
Freq: B, Name: Signal, dtype: float64


Date
2007-07-13   -0.025097
2007-07-16    0.029043
2007-07-17    0.002566
2007-07-18    0.023672
2007-07-19   -0.048125
                ...   
2021-11-11   -0.057128
2021-11-12   -0.077576
2021-11-15    0.012277
2021-11-16   -0.007277
2021-11-17    0.000000
Freq: B, Name: VIX_ret_close, Length: 3744, dtype: float64

In [699]:
#  Validation on missing data on VIX
vix_ret=vix.pct_change()
vix_ret[vix_ret>=threshold].index
vix_ret.shape


# How many values of the vix we missed due to missing data on other series
compare=pd.concat([XY.loc[XY["Signal"]==1, 'VIX_ret_close'],vix_ret[vix_ret>=threshold] ], axis=1)
missing_dates=compare.loc[compare["VIX_ret_close"]!=compare["^VIX"]]
missing_dates=missing_dates.index
missing_dates
if parameter_tuning_mode == True:
    vix[missing_dates]

In [700]:
# Define the target set y using the Signal column
y = XY["Signal"]
# Display a sample of y
if parameter_tuning_mode == True:
    y
#pd.concat([vix_ret,y], axis=1).head(20)

In [701]:
# Outputs for the model tuning

if parameter_tuning_mode == True:
    display(y.isna().sum())
    display(y.shape)
    display(XY.shape)
    display(XY.drop(columns=["Signal"]).isna().sum())
    display(XY.drop(columns=['Signal']).shift().isna().sum())
    display(XY.drop(columns=['Signal']).shift().dropna().shape)

0

(4013,)

(4013, 109)

VIX_close          0
VIX_ret_close      1
spy_close          0
USDJPY=X_close     0
NKD=F_close        0
                  ..
Friday            33
Monday            33
Thursday          33
Tuesday           33
Wednesday         33
Length: 108, dtype: int64

VIX_close          1
VIX_ret_close      2
spy_close          1
USDJPY=X_close     1
NKD=F_close        1
                  ..
Friday            33
Monday            33
Thursday          33
Tuesday           33
Wednesday         33
Length: 108, dtype: int64

(3476, 108)

In [702]:
# Set up of X, y and outputs for the model tuning

XY_modified = XY.shift().dropna()
if parameter_tuning_mode == True:
    display(XY_modified.shape)

y = XY_modified["Signal"].shift(-1)

X = XY_modified

if parameter_tuning_mode == True:
    display(y.shape)
    display(X.shape)
    display(y.isna().sum())
    display(X.isna().sum())


(3476, 109)

(3476,)

(3476, 109)

1

VIX_close         0
VIX_ret_close     0
spy_close         0
USDJPY=X_close    0
NKD=F_close       0
                 ..
Monday            0
Thursday          0
Tuesday           0
Wednesday         0
Signal            0
Length: 109, dtype: int64

In [703]:
# Review the features DataFrame
if parameter_tuning_mode == True:
    X.head()

In [704]:
# Review of correlations
if parameter_tuning_mode == True:
    X.corr()

# Split of data in Train and Test (I)

In [705]:
# Split data into training and testing subsets

def split_training_test_data(X, y):
    """
    This function split the preprocessed data of a time series into two windows: a training window and a testing window,
    Its give back the features and signals divded accordingly
    
    Args:
    X: a pandas dataframe with the features in its columns, using a datetime index
    y: a pandas dataframe with the signal, using a datetime index
    
    Return:
    Two pandas dataframes and two series in the following order:
    X_train: a pandas dataframe with the features of the train window
    y_train: a pandas series with the signals of the train window 
    X_test: a pandas dataframe with the featires in the test window
    y_test: a pandas series with the signals of the train window 
    
    """
    
    

    training_begin = X.index.min()
    training_end = X.index.min() + DateOffset(months=training_period_months)

    X_train = X.loc[training_begin:training_end]
    y_train = y.loc[training_begin:training_end]

    X_test = X.loc[training_end + DateOffset(days=1):]
    y_test = y.loc[training_end + DateOffset(days=1):]

    if parameter_tuning_mode == True:
        print(f"Training dates: {training_begin} to {training_end}")
        display(y_train.value_counts())
        display(y_test.shape)
        display(X_test.shape)
        display(X_train.shape)
        display(y_train.shape)
        display(X_train.tail(1))
        display(X_test.head(1))
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = split_training_test_data(X, y)



Training dates: 2007-07-03 00:00:00 to 2017-07-03 00:00:00


0.0    1308
1.0    1129
Name: Signal, dtype: int64

(1039,)

(1039, 109)

(2437, 109)

(2437,)

,VIX_close,VIX_ret_close,spy_close,USDJPY=X_close,NKD=F_close,LQD_close,BMY_close,FXI_close,FEZ_close,^FTSE_close,...,120_spy_rolling_volatility,180_spy_rolling_volatility,200_spy_rolling_volatility,260_spy_rolling_volatility,Friday,Monday,Thursday,Tuesday,Wednesday,Signal
Date,,,,,,,,,,,,,,,,,,,,,
2017-07-03,11.18,-0.022727,224.135193,112.009003,20105.0,105.289787,48.687695,35.960217,34.746212,7312.700195,...,0.004487,0.004755,0.004951,0.005969,1.0,0.0,0.0,0.0,0.0,0.0


,VIX_close,VIX_ret_close,spy_close,USDJPY=X_close,NKD=F_close,LQD_close,BMY_close,FXI_close,FEZ_close,^FTSE_close,...,120_spy_rolling_volatility,180_spy_rolling_volatility,200_spy_rolling_volatility,260_spy_rolling_volatility,Friday,Monday,Thursday,Tuesday,Wednesday,Signal
Date,,,,,,,,,,,,,,,,,,,,,
2017-07-06,11.07,-0.013369,225.034332,112.961998,20100.0,105.237251,49.581833,36.186615,34.809441,7367.600098,...,0.004486,0.004747,0.004898,0.005967,0.0,0.0,0.0,0.0,1.0,0.0


In [706]:
# Scaling of the data

def standard_scale(X_train, X_test):
    """
    This function apply standard scaling to a divided set of features divided as train and test data
    
    Args:
    The tow dataframes:
    X_train: a pandas dataframe with features of the training window
    X_test: a pandas dataframe with features of the test window
    
    Return:
    Two arrays coming from the original dataframes after applying StandardScaler(), where the standarization is made using the X_train features
    """
    # Create a StandardScaler instance
    scaler =  StandardScaler() # MinMaxScaler() #
 
    # Apply the scaler model to fit the X-train data
    X_scaler = scaler.fit(X_train)

    # Transform the X_train and X_test DataFrames using the X_scaler
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    if parameter_tuning_mode == True:
        display(X_train_scaled.shape)
        display(X_test_scaled.shape)
    return X_train_scaled, X_test_scaled
    
X_train_scaled, X_test_scaled = standard_scale(X_train, X_test)


(2437, 109)

(1039, 109)

### X lags: calculate principal components of Train data, and lagged them 
number of components tuned to 4, and considered lags: t=5 days

In [707]:
# Calculation of Principal Components

def adaboost_pca(X_train, X_test):
    
    """
    This function calculates the principal components of an X features matrix, already divided in a train and a test set.
    
    Args:
    X_train: a pandas dataframe with the training set of features
    X_test: a pandas dataframe with the test set of features
    
    
    Returns:
    An X pandas dataframe of features, with the train and test samples concatenated, which correspond
    to the principal components of the original data, calculated using the transformation calculated from the train set of data.
    """
    
    # Initiate and calculate principal components transformation based on the train data
    pca = PCA(n_components = num_pca_components)
    pca.fit(X_train)
    
    # Calculate train and test principal components using the trained model
    principal_components_train = pca.transform(X_train)
    principal_components_test  = pca.transform(X_test)
    
    # Name principal components columns properly
    pca_column_list = []
    for i in range(1, num_pca_components+1):
        pca_column_list.append(f"pca{i}")

    #Concatenate train and test principal components in one dataframe called principal_components_train_test_df
    principal_components_train_test = np.concatenate((principal_components_train, principal_components_test), axis = 0)
    principal_components_train_test_df = pd.DataFrame(data = principal_components_train_test, columns = pca_column_list, index = X.index)
    if parameter_tuning_mode == True:
        display(sum(pca.explained_variance_ratio_))
        display(principal_components_train_test_df.shape)
        display(principal_components_train_test_df.head(5))
    return principal_components_train_test_df
principal_components_train_test_df = adaboost_pca(X_train_scaled, X_test_scaled)

0.9179003038551657

(3476, 40)

,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10,...,pca31,pca32,pca33,pca34,pca35,pca36,pca37,pca38,pca39,pca40
Date,,,,,,,,,,,,,,,,,,,,,
2007-07-03,-3.554289,1.468720,0.886001,3.468464,-2.221634,2.629429,-0.404885,0.059725,-1.037807,0.342902,...,-0.581533,-0.102422,0.441183,-1.319024,0.724789,0.360257,-0.199974,0.111825,-0.484335,-0.784961
2007-07-06,-3.599896,0.156004,0.934164,4.447028,-2.385316,2.388547,1.187554,0.454223,1.591849,-0.571203,...,-0.181208,0.082251,1.071254,-1.054137,-0.810885,-0.231111,-0.576947,-0.240110,1.535039,-0.151689
2007-07-09,-4.302785,1.253188,0.663482,3.464827,-2.142664,2.384773,0.029529,0.472510,0.763927,0.566853,...,0.176428,-0.784381,-0.007645,-0.573548,0.364488,-0.026544,-0.293023,0.496375,0.269331,-0.249880
2007-07-10,-4.231141,-0.381648,0.470708,3.692985,-2.043560,3.269258,-0.103054,-0.058665,-0.178126,0.720844,...,-0.537388,-0.919348,-0.037343,-0.348185,0.626497,0.901877,-0.545512,0.066974,-0.473096,0.146036
2007-07-11,-2.291334,-4.098464,0.988929,4.379990,-1.727683,2.338579,0.662894,-0.675548,-0.220574,-0.517900,...,0.562341,-0.161830,0.651590,-0.102312,-0.856938,-0.200287,-0.690567,0.006407,-0.313137,-0.009233


In [708]:
# Generation of lag principal components to include historical movements of the data in the model. We call this set LAG.

def create_pca_lag(principal_components_train_test_df, shift_amount):
    """
    This function creates a new dataframe by taking the first 3 first principal components (in the first three columns) and shifting them in a desire number
    
    Args:
    principal_components_train_test_df: a pandas dataframe 
    shift_amount: the number of positions to shift the first three columnms
    
    Return:
    A pandas dataframe containing the shifted 3 first columns of the dataframe,
    with column names 'pca1_lag1','pca2_lag1','pca3_lag1'
    """
    X_pc_lag = principal_components_train_test_df.iloc[:,0:(number_of_pca_lag_component_to_include-1)]
    if parameter_tuning_mode == True:
        display(X_pc_lag.shape)

    X_pc_lag.columns = ['pca1_lag1','pca2_lag1','pca3_lag1']
    X_pc_lag = X_pc_lag.shift(shift_amount)

    if parameter_tuning_mode == True:
        print(X_pc_lag)
        X_pc_lag.shape
    return X_pc_lag

# Shift the LAG components by 1
X_pca_lag1 = create_pca_lag(principal_components_train_test_df, 1)

(3476, 3)

            pca1_lag1  pca2_lag1  pca3_lag1
Date                                       
2007-07-03        NaN        NaN        NaN
2007-07-06  -3.554289   1.468720   0.886001
2007-07-09  -3.599896   0.156004   0.934164
2007-07-10  -4.302785   1.253188   0.663482
2007-07-11  -4.231141  -0.381648   0.470708
...               ...        ...        ...
2021-09-27  -7.605371   1.554771   8.783984
2021-09-28  -7.847541  -1.259617   8.382125
2021-09-29  -7.774759  -1.996421   8.391289
2021-09-30  -5.069053  -6.095363   9.583118
2021-10-01  -6.704071  -0.682711   8.399857

[3476 rows x 3 columns]


In [709]:
# Shift the LAG components by 2    
X_pca_lag2 = create_pca_lag(principal_components_train_test_df, 2)

(3476, 3)

            pca1_lag1  pca2_lag1  pca3_lag1
Date                                       
2007-07-03        NaN        NaN        NaN
2007-07-06        NaN        NaN        NaN
2007-07-09  -3.554289   1.468720   0.886001
2007-07-10  -3.599896   0.156004   0.934164
2007-07-11  -4.302785   1.253188   0.663482
...               ...        ...        ...
2021-09-27  -7.196858   1.594943   8.720459
2021-09-28  -7.605371   1.554771   8.783984
2021-09-29  -7.847541  -1.259617   8.382125
2021-09-30  -7.774759  -1.996421   8.391289
2021-10-01  -5.069053  -6.095363   9.583118

[3476 rows x 3 columns]


In [710]:
# Shift the LAG components by 3
X_pca_lag3 = create_pca_lag(principal_components_train_test_df, 3)

(3476, 3)

            pca1_lag1  pca2_lag1  pca3_lag1
Date                                       
2007-07-03        NaN        NaN        NaN
2007-07-06        NaN        NaN        NaN
2007-07-09        NaN        NaN        NaN
2007-07-10  -3.554289   1.468720   0.886001
2007-07-11  -3.599896   0.156004   0.934164
...               ...        ...        ...
2021-09-27  -6.951441  -0.005681   8.442270
2021-09-28  -7.196858   1.594943   8.720459
2021-09-29  -7.605371   1.554771   8.783984
2021-09-30  -7.847541  -1.259617   8.382125
2021-10-01  -7.774759  -1.996421   8.391289

[3476 rows x 3 columns]


In [711]:
# Shift the LAG Components by 4
X_pca_lag4 = create_pca_lag(principal_components_train_test_df, 4)

(3476, 3)

            pca1_lag1  pca2_lag1  pca3_lag1
Date                                       
2007-07-03        NaN        NaN        NaN
2007-07-06        NaN        NaN        NaN
2007-07-09        NaN        NaN        NaN
2007-07-10        NaN        NaN        NaN
2007-07-11  -3.554289   1.468720   0.886001
...               ...        ...        ...
2021-09-27  -4.936636  -6.996283   9.534783
2021-09-28  -6.951441  -0.005681   8.442270
2021-09-29  -7.196858   1.594943   8.720459
2021-09-30  -7.605371   1.554771   8.783984
2021-10-01  -7.847541  -1.259617   8.382125

[3476 rows x 3 columns]


In [712]:
# Shift the LAG Components by 5
X_pca_lag5 = create_pca_lag(principal_components_train_test_df, 5);

(3476, 3)

            pca1_lag1  pca2_lag1  pca3_lag1
Date                                       
2007-07-03        NaN        NaN        NaN
2007-07-06        NaN        NaN        NaN
2007-07-09        NaN        NaN        NaN
2007-07-10        NaN        NaN        NaN
2007-07-11        NaN        NaN        NaN
...               ...        ...        ...
2021-09-27  -6.894069  -2.948111   8.182486
2021-09-28  -4.936636  -6.996283   9.534783
2021-09-29  -6.951441  -0.005681   8.442270
2021-09-30  -7.196858   1.594943   8.720459
2021-10-01  -7.605371   1.554771   8.783984

[3476 rows x 3 columns]


In [713]:
def concatenate_lags(X_pc_lag1, X_pc_lag2, X_pc_lag3, X_pc_lag4, X_pc_lag5):
    X_pc_lags=pd.concat([X_pc_lag1, 
                         X_pc_lag2, 
                         X_pc_lag3, 
                         X_pc_lag4, 
                         X_pc_lag5], 
                         axis=1
    )
    
    if parameter_tuning_mode == True:
        X_pc_lags.shape
    return X_pc_lags

X_pc_lags = concatenate_lags(X_pca_lag1, X_pca_lag2, X_pca_lag3, X_pca_lag4, X_pca_lag5)

In [714]:
# Concatenation of all variables in X_pc, storing current variables plus lagged principal components

def combine_train_test(X_train, X_test):
    """
    This function concantenate the train and test arrays, and apply the proper index, to get back to an X dataframe
    
    Args:
    X_train, X_test: arrays to concatenate, wich should have the same number of columns
    
    Return:
    The X dataframe as a pandas dataframe, and the index as the last X dataframe of features
    """
    
    X_combined = np.concatenate([X_train, X_test], axis = 0)
    X_combined = pd.DataFrame(data = X_combined, index=X.index)
    return X_combined

def concatenate_with_pca_lags(X_raw, X_pc_lags):
    """
    This function concatenates all the sources of data: features and lags principal compoents.
    It also eliminates NaNs due to lag
    
    Args:
    X_raw: the combination of X_train and X_test features, excluding principal compoentns
    X_pc_lags: the 3 main principal components of the set of features, lagged in 1, 2, 3, 4 and 5 days
    
    Returns:
    A dataframe containing the concatenation of all features and principal components without missing values, with the proper datetime index
    """
    
    X_pc = pd.concat([X_raw, X_pc_lags], axis=1)

    if parameter_tuning_mode == True:
        print(X_pc.shape)
    return X_pc

X_scaled_df = combine_train_test(X_train_scaled, X_test_scaled)
X_pc = concatenate_with_pca_lags(X_scaled_df, X_pc_lags)

(3476, 124)


In [715]:
# Displaying outputs for tuning
if parameter_tuning_mode == True:
    print(f"principal_components_train_test_df.shape: {principal_components_train_test_df.shape}, {principal_components_train_test_df.index.min()}, {principal_components_train_test_df.index.max()} ")
    print(f"X_pc_lags.shape: {X_pc_lags.shape}, {X_pc_lags.index.min()}, {X_pc_lags.index.max()} ")
    print(f"X_pc.shape: {X_pc.shape}, {X_pc.index.min()}, {X_pc.index.max()} ")
    print(f"y.shape: {y.shape}, {y.index.min()}, {y.index.max()}")

principal_components_train_test_df.shape: (3476, 40), 2007-07-03 00:00:00, 2021-10-01 00:00:00 
X_pc_lags.shape: (3476, 15), 2007-07-03 00:00:00, 2021-10-01 00:00:00 
X_pc.shape: (3476, 124), 2007-07-03 00:00:00, 2021-10-01 00:00:00 
y.shape: (3476,), 2007-07-03 00:00:00, 2021-10-01 00:00:00


In [716]:
# Elimination of missing data in principal components

def eliminate_nans_in_pca_data(X_pc, y):
    X_pc = X_pc[5:-1]
    y = y[5:-1]

    if parameter_tuning_mode == True:
        display(X_pc.shape)
        display(y.shape)
    return X_pc, y

X_pc, y = eliminate_nans_in_pca_data(X_pc, y)

(3470, 124)

(3470,)

In [717]:
# Redefinition of X and y with extended X, to apply convention to feature data
X = X_pc

if parameter_tuning_mode == True:
    print(X.shape)
    y.shape
    
column_names = [*X_train.columns, *X_pc_lags.columns]
X.columns = column_names 


(3470, 124)


# Split the data in train and test

In [718]:
# Split of data in train and test, applying temporal window function that respect time series order, which is defined above in cell 177

X_train, y_train, X_test, y_test = split_training_test_data(X, y)
X_train_scaled, X_test_scaled = standard_scale(X_train, X_test)

Training dates: 2007-07-12 00:00:00 to 2017-07-12 00:00:00


0.0    1310
1.0    1127
Name: Signal, dtype: int64

(1033,)

(1033, 124)

(2437, 124)

(2437,)

,VIX_close,VIX_ret_close,spy_close,USDJPY=X_close,NKD=F_close,LQD_close,BMY_close,FXI_close,FEZ_close,^FTSE_close,...,pca3_lag1,pca1_lag1,pca2_lag1,pca3_lag1,pca1_lag1,pca2_lag1,pca3_lag1,pca1_lag1,pca2_lag1,pca3_lag1
Date,,,,,,,,,,,,,,,,,,,,,
2017-07-12,-0.996594,-0.289432,2.088699,1.066723,1.762453,1.534014,1.141324,1.013187,1.247239,1.66366,...,3.446117,-5.775234,-3.066633,3.602243,-6.009522,1.017205,3.759007,-6.054628,-0.689013,3.377002


,VIX_close,VIX_ret_close,spy_close,USDJPY=X_close,NKD=F_close,LQD_close,BMY_close,FXI_close,FEZ_close,^FTSE_close,...,pca3_lag1,pca1_lag1,pca2_lag1,pca3_lag1,pca1_lag1,pca2_lag1,pca3_lag1,pca1_lag1,pca2_lag1,pca3_lag1
Date,,,,,,,,,,,,,,,,,,,,,
2017-07-13,-1.056718,-0.738492,2.12704,1.049659,1.766338,1.565601,1.12488,1.17171,1.297385,1.771948,...,3.563171,-6.375655,1.305575,3.446117,-5.775234,-3.066633,3.602243,-6.009522,1.017205,3.759007


(2437, 124)

(1033, 124)

In [719]:
# Setting unique columns names to be able to apply random over sample model
column_name_list = []
for i in range(0, len(X.columns)):
    column_name_list.append(f"f_{i}")
X_train_unique_columns = X_train.copy()
X_train_unique_columns.columns = column_name_list


In [720]:
# Random oversample was applied since depending on the Threshold (above which return we are predicting), the sample can get highly unbalanced
# For the case of threshold = 0

def random_over_sample(X_train, y_train):
    # Use RandomOverSampler to resample the dataset using random_state=1
    ros = RandomOverSampler(random_state = 1)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

    if parameter_tuning_mode == True:
        display(y_train_resampled.value_counts())
    return X_train_resampled, y_train_resampled

X_train_resampled, y_train_resampled = random_over_sample(X_train_scaled, y_train)

0.0    1310
1.0    1310
Name: Signal, dtype: int64

# Adaboost Model Estimation

In [744]:
# Instance AdaBoost
# Initiate the model instance
base = DecisionTreeClassifier(max_depth=2  )  #max_depth)
adaboost_model=AdaBoostClassifier(base_estimator=base
                                  ,n_estimators=6  #adaboost_estimators
                                  ,learning_rate= learning_rate_adaboost )
adaboost_model

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2),
                   learning_rate=1, n_estimators=6)

In [745]:
# Tunning for the model
if parameter_tuning_mode == True:
    display(X_train_resampled.shape)
    display(y_train_resampled.shape)
    display(X_test_scaled.shape)
    display(y_test.shape)

(2620, 124)

(2620,)

(1033, 124)

(1033,)

In [746]:
# Fit the model 
adaboost_model =adaboost_model.fit(X_train_resampled, y_train_resampled)

pred_adaboost=adaboost_model.predict(X_test_scaled)

In [747]:
if demo_mode == False and parameter_tuning_mode == True:
    display(np.any(np.isnan(y_test)))
    display(np.all(np.isfinite(y_test)))
    display(np.any(np.isnan(pred_adaboost)))
    display(np.all(np.isfinite(pred_adaboost)))
    display(y_test.shape)
    display(pred_adaboost.shape)

False

True

False

True

(1033,)

(1033,)

In [767]:
# Use a classification report to evaluate the model using the predictions and testing data
adaboost_report=classification_report(y_test, pred_adaboost)

# Print the classification report
print("         AdaBoost Classification Report")
print(adaboost_report)


         AdaBoost Classification Report
              precision    recall  f1-score   support

         0.0       0.61      0.68      0.65       569
         1.0       0.55      0.47      0.51       464

    accuracy                           0.59      1033
   macro avg       0.58      0.58      0.58      1033
weighted avg       0.58      0.59      0.58      1033



# Analysis of feature importance in the AdaBoost model

In [768]:
# Analysis of importance of the difference variables
# get importance
adaboost_importance_coeficients=adaboost_model.feature_importances_

feature_importance_df=pd.Series(
                                adaboost_importance_coeficients, 
                                index=X.columns)

# Output of all levels
for i,v in enumerate(adaboost_importance_coeficients):
    if v !=0:
        print(f"Feature: {i}, {X.columns[i]}, Score: {v}" )
        

# Display of only features that impacted the model
#n_important_features=feature_importance_df.loc[feature_importance_df>0].shape[0]
#feature_importance_df.loc[feature_importance_df>0].hvplot(
#                                            kind='barh', 
#                                            height=500,
#                                            title= f"{n_important_features} Features relevant for the VIX Prediction Model")

Feature: 24, PLD_returns, Score: 0.04996908401048105
Feature: 26, FEZ_returns, Score: 0.07878449193933011
Feature: 31, BIO=F_returns, Score: 0.05257015603530681
Feature: 34, AAPL_volume, Score: 0.05381350708146323
Feature: 50, USDJPY=X_garch, Score: 0.06090673862210311
Feature: 51, ^TNX_garch, Score: 0.05572682751038066
Feature: 52, ZF=F_garch, Score: 0.04946887502518976
Feature: 53, NQ=F_garch, Score: 0.08348944588134566
Feature: 54, NKD=F_garch, Score: 0.05400808212213944
Feature: 77, NQ=F_return_squared, Score: 0.06165541257128687
Feature: 80, AAPL_return_squared, Score: 0.06447952006800332
Feature: 86, FEZ_return_squared, Score: 0.06288407557472239
Feature: 90, gld_return_squared, Score: 0.060088428509220436
Feature: 91, slv_return_squared, Score: 0.08769772628714594
Feature: 103, Friday, Score: 0.04881119103571795
Feature: 105, Thursday, Score: 0.03436602974960306
Feature: 116, pca2_lag1, Score: 0.04128040797656025


# Profitability Analysis
The large size of VIX returns would make a huge compounding effect if it were possible to bet on it and track its performance,.
As can be seen below, the average size of daily returns are around 5%, with returns that can get as large as 115% in one day. The median is 

In this section we will compare the results of betting on the model applying an imaginary strategy of betting at the open of the day, and close the bet at the end of the day. We will assume that the approximately the open level will be very similar than the close level of the previous day.

In [769]:
# VIX size of returns (in absolute value and %)
vix_return_statistics=abs(100 * security_returns_df['^VIX']).describe()
display(vix_return_statistics)

count    4012.000000
mean        5.313423
std         5.992332
min         0.000000
25%         1.521798
50%         3.750664
75%         7.143602
max       115.597925
Name: ^VIX, dtype: float64

### In-sample analysis: Return on $1 invested on training data window

In [770]:
# Results comnparison

# Profitability on the train window
fit_train= adaboost_model.predict(X_train_scaled)
fit_train_df= pd.DataFrame(fit_train, index=X_train.index)

fit_train_df.hvplot()

y_train_df=pd.DataFrame(y_train, index=X_train.index)
y_train_df

vix_returns_train_df=vix_ret[y_train_df.index.min():y_train_df.index.max()]
vix_returns_train_df = vix_returns_train_df[y_train_df.index]


results_train_df=pd.concat([vix_returns_train_df, y_train_df, fit_train_df], axis=1)

results_train_df.columns=['VIX Return', 'Correct Signal', 'Fit Signal']

predicted_return=results_train_df['VIX Return']*results_train_df['Fit Signal']
max_return=results_train_df['VIX Return']*results_train_df['Correct Signal']


results_train_df=pd.concat([results_train_df, predicted_return, max_return], axis=1)
results_train_df.columns=['VIX Return', 'Correct Signal', 'Predicted Signal', 'Fit Return', 'Max Return']


return_of_one_dollar_in_train_window_df=(1+results_train_df[['VIX Return','Fit Return']]).cumprod()
return_of_one_dollar_in_train_window_df.columns=['VROI', 'ROI Model (in sample)']

profitability_train_plot=return_of_one_dollar_in_train_window_df.hvplot(
                                            title="In-Sample Growth of $1 initial Investment in Daily Trade Strategy on VIX",
                                            ylabel="Dollars $" ,
                                            width=1000
)

profitability_train_plot

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

### Out-of--sample analysis: Return on $1 invested on training data window

In [772]:
# Results comnparison

# Profitability on the test window

# Out-of-sample Predictions 
prediction_test= adaboost_model.predict(X_test_scaled)
prediction_test_df= pd.DataFrame(prediction_test, index=X_test.index)

# Out-of-sample signals (1s or 0s) based on actual returns of the VIX
y_test_df=pd.DataFrame(y_test, index=X_test.index)
y_test_df

# VIX returns in the test window
vix_returns_df=vix_ret[y_test_df.index.min():y_test_df.index.max()]
vix_returns_test_df = vix_returns_df[y_test_df.index]

# Combination of VIX Returns, signals and predictions
results_test_df=pd.concat([vix_returns_test_df, y_test_df, prediction_test_df], axis=1)
results_test_df.columns=['VIX Return', 'Correct Signal', 'Predicted Signal']

# Predicted returns
predicted_return=results_test_df['VIX Return']*results_test_df['Predicted Signal']
max_return=results_test_df['VIX Return']*results_test_df['Correct Signal']

# DataFrame with out-of-sample results for comparison
results_test_df=pd.concat([results_test_df, predicted_return, max_return], axis=1)
results_test_df.columns=['VIX Return', 'Correct Signal', 'Predicted Signal', 'Predicted Return', 'Max Return']

return_of_one_dollar_in_test_window_df=(1+results_test_df[['VIX Return', 'Predicted Return']]).cumprod()
return_of_one_dollar_in_test_window_df.columns=['VIX Cummulative return', ' Model Cummulative Return (out of sample)']


#Plot with out of sample return on a 1 dollar investment for the VIX, and the daily bet strategy
profitability_test_plot=return_of_one_dollar_in_test_window_df.hvplot(
                         title='Out-Of-Sample Growth of $1 initial Investment in Daily Trade Strategy on VIX ',
                         ylabel= "Dollars $",
                         width=1000
                        )
profitability_test_plot

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [773]:
# Results in prediction of daily returns
min_return=threshold
results_test_for_plot_df=results_test_df[abs(results_test_df['VIX Return'])>min_return]*100

results_test_for_plot_df.hvplot(
                    y=['VIX Return', 'Predicted Return'],
                    title= "Out-of-sample predictions of VIX return",
                    width=1000,
                    ylabel='Daily Return (%)'


)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [774]:
#Histogram of returns out of sample
results_test_for_plot_df.hvplot.hist(
    ['VIX Return','Predicted Return'],
     title= "Out-of-sample VIX returns predictions"
)

:NdOverlay   [Element]
   :Histogram   [Predicted Return]   (Predicted Return_count)

### Analysis of good and bad predictions

In [775]:
# Analysis of the good and bad out-of-sample predictions
good_predictions=results_test_df[results_test_df['Correct Signal']==results_test_df['Predicted Signal']]

bad_predictions=results_test_df[results_test_df['Correct Signal']!=results_test_df['Predicted Signal']]


In [776]:
good=good_predictions.hvplot.hist(
    ['VIX Return','Predicted Return'],
     title= "Out-of-sample VIX Returns of Good Predictions"
)

bad=bad_predictions.hvplot.hist(
    ['VIX Return','Predicted Return'],
     title= "Out-of-sample VIX Returns on Bad Predictions"
)
good+bad

:Layout
   .NdOverlay.I  :NdOverlay   [Element]
      :Histogram   [Predicted Return]   (Predicted Return_count)
   .NdOverlay.II :NdOverlay   [Element]
      :Histogram   [Predicted Return]   (Predicted Return_count)

In [777]:
# Good prediction statistics
good_predictions[['VIX Return', 'Predicted Return']].describe()

,VIX Return,Predicted Return
count,608.000000,608.000000
mean,-0.011864,0.022926
std,0.095749,0.075151
min,-0.233735,-0.027680
25%,-0.059516,-0.000000
50%,-0.019590,-0.000000
75%,0.013754,0.012751
max,1.155979,1.155979


In [778]:
# Bad prediction statistics
bad_predictions[['VIX Return', 'Predicted Return']].describe()

,VIX Return,Predicted Return
count,425.000000,425.000000
mean,0.024212,-0.016170
std,0.086785,0.033656
min,-0.205029,-0.205029
25%,-0.024656,-0.023937
50%,0.007168,0.000000
75%,0.056901,0.000000
max,0.479507,0.197941


In [779]:
# Box Plot predictions for good and bad predictions
good_pred=good_predictions['Predicted Return']
bad_pred =bad_predictions['Predicted Return']

predictions_comparison_df=pd.concat([good_predictions['Predicted Return'],bad_predictions['Predicted Return']], axis=1, ignore_index=True )*100
predictions_comparison_df.hvplot(kind='box',
                                height=800,
                                ylabel='Return (%)',
                                #clabel=['Good Predictions', 'Bad Predictions'],
                                cmap=['blue','red'],
                                title="Out-of-sample Good and Bad Returns Resulting from Model Predictions Box Plots")





#df = pd.DataFrame(np.random.randn(20), columns=['Value'])
#df['Source'] = ['Preds'] *10 +['Real'] * 10
#df['Item'] = ['item1'] *5 + ['item2']*5 + ['item1'] *5 + ['item2']*5
#df.hvplot.box(y='Value', by=['Item', 'Source'])

:BoxWhisker   [Variable]   (value)

# OUTPUTS FOR TUNNING

In [542]:
def prepare_features(XY, pca_components):
    XY_modified = XY.shift().dropna()
    y = XY_modified["Signal"].shift(-1)
    X = XY_modified
    pca = PCA(n_components = pca_components)
    principal_components = pca.fit_transform(X)
    
    pca_column_list = []
    for i in range(1, pca_components+1):
        pca_column_list.append(f"pca{i}")

    principal_components_train_test_df = pd.DataFrame(data = principal_components, columns = pca_column_list, index = XY_modified.index)
    X_pca_lag1 = create_pca_lag1(principal_components_train_test_df)
    X_pca_lag2 = create_pca_lag2(principal_components_train_test_df)
    X_pca_lag3 = create_pca_lag3(principal_components_train_test_df)
    X_pca_lag4 = create_pca_lag4(principal_components_train_test_df)
    X_pca_lag5 = create_pca_lag5(principal_components_train_test_df)
        
    X_pc_lags = concatenate_lags(X_pca_lag1, X_pca_lag2, X_pca_lag3, X_pca_lag4, X_pca_lag5)
    X_pc = concatenate_pca_with_lags(principal_components_train_test_df, X_pc_lags)
    X, y = eliminate_nans_in_pca_data(X_pc, y)
    
    X_train, y_train, X_test, y_test = split_training_test_data(X, y)
    X_train_resampled, y_train_resampled = random_over_sample(X_train, y_train)
    X_train_scaled, X_test_scaled = standard_scale(X_train_resampled, X_test)
    #principal_components_train_test
    if parameter_tuning_mode == True:
        display(sum(pca.explained_variance_ratio_))
        display(principal_components_train_test_df.shape)
    return X_train_scaled, X_test_scaled, y_train_resampled, y_test

In [638]:
if run_multiple_tuning_iterations == True: 
    for num_estimators in range (1,20, 1):
        for depth in range(1,4,1):
            base = DecisionTreeClassifier(max_depth=depth)
            adaboost_model=AdaBoostClassifier(base_estimator=base, 
                                  n_estimators=num_estimators, 
                                  learning_rate= learning_rate_adaboost )

            # Fit the model 
            adaboost_model = adaboost_model.fit(X_train_resampled, y_train_resampled)
            pred_adaboost = adaboost_model.predict(X_test)
            # Use a classification report to evaluate the model using the predictions and testing data
            adaboost_report=classification_report(y_test, pred_adaboost)

            #if num_estimators % 10 == 0 and num_components == 88:
            #    print(f"components {num_components} esimators {num_estimators}")
            #    print(f"f1 score 0 {f1_score(y_test, pred_adaboost, pos_label=0)} f1 score 1 {f1_score(y_test, pred_adaboost, pos_label=1)}")
            #    print(f"accuracy {accuracy_score(y_test, pred_adaboost)}")
            #    print(adaboost_report)
            f1_score_1 = f1_score(y_test, pred_adaboost, pos_label=1)
            f1_score_0 = f1_score(y_test, pred_adaboost, pos_label=0)
            recall_score_1 = recall_score(y_test, pred_adaboost, pos_label=1)
            recall_score_0 = recall_score(y_test, pred_adaboost, pos_label=0)
            accuracy_score_model = accuracy_score(y_test, pred_adaboost)
            if  accuracy_score_model >= .55 and f1_score_1 >= .50 and f1_score_0 >= .50 and recall_score_1 >= .50 and recall_score_0 >= .50:
                print(f"estimators {num_estimators}, max tree deph: {depth}")
                # print(f"variance explained {sum(pca.explained_variance_ratio_)}")
                # Print the classification report
                print("         AdaBoost Classification Report")
                print(adaboost_report)
            print(f"Trying deph {depth} and n_estimators {num_estimators}")

Trying deph {deph} and n_estimators {num_estimators}
Trying deph {deph} and n_estimators {num_estimators}
Trying deph {deph} and n_estimators {num_estimators}
estimators 2, max tree deph: 1
         AdaBoost Classification Report
              precision    recall  f1-score   support

         0.0       0.63      0.70      0.66       569
         1.0       0.57      0.50      0.53       464

    accuracy                           0.61      1033
   macro avg       0.60      0.60      0.60      1033
weighted avg       0.60      0.61      0.60      1033

Trying deph {deph} and n_estimators {num_estimators}
Trying deph {deph} and n_estimators {num_estimators}
Trying deph {deph} and n_estimators {num_estimators}
estimators 3, max tree deph: 1
         AdaBoost Classification Report
              precision    recall  f1-score   support

         0.0       0.63      0.70      0.66       569
         1.0       0.57      0.50      0.53       464

    accuracy                           0.61      

In [451]:
# Number of estimators? 
if run_multiple_tuning_iterations == True:
    for n in range (50,200, 10):
        # Instance AdaBoost
        # Initiate the model instance
        adaboost_model=AdaBoostClassifier(n_estimators=n)

        # Fit the model 
        adaboost_model =adaboost_model.fit(X_train_scaled, y_train)
        pred_adaboost=adaboost_model.predict(X_test_scaled)
        print (n)
        # Use a classification report to evaluate the model using the predictions and testing data
        adaboost_report=classification_report(y_test, pred_adaboost)

        # Print the classification report
        print("         AdaBoost Classification Report")
        print(adaboost_report)
#120 highest 1-recall
#150 good overall accuracy, but lower 1-recall


50
         AdaBoost Classification Report
              precision    recall  f1-score   support

         0.0       0.61      0.57      0.59       569
         1.0       0.51      0.54      0.53       464

    accuracy                           0.56      1033
   macro avg       0.56      0.56      0.56      1033
weighted avg       0.56      0.56      0.56      1033

60
         AdaBoost Classification Report
              precision    recall  f1-score   support

         0.0       0.60      0.55      0.58       569
         1.0       0.50      0.55      0.53       464

    accuracy                           0.55      1033
   macro avg       0.55      0.55      0.55      1033
weighted avg       0.56      0.55      0.55      1033

70
         AdaBoost Classification Report
              precision    recall  f1-score   support

         0.0       0.61      0.51      0.55       569
         1.0       0.50      0.61      0.55       464

    accuracy                           0.55      1033

#### Future enhancements:

* Clear variables that are not important
* Try increasing the deph of the AdaBoost model
* Include more features
* X8: use a function to generate the rolling volatilities, and set the number on the column name